# Basketball "Analysis"

Kwabena Aduse-Poku

## The data

Blah blah blah heres some data

But first, necessary imports

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
res = requests.get('http://www.prosportstransactions.com/basketball/Search/SearchResults.php?Player=&Team=&BeginDate=2010-10-01&EndDate=2022-05-09&NBADLChkBx=yes&Submit=Search')
len(res.text)

51518

In [3]:
soup = BeautifulSoup(res.text, 'html.parser')
for table in soup.find_all('table'):
    print(table.get('class'))

['datatable', 'center']
None


In [4]:
gleague_table = soup.find('table', class_ ='datatable')

df = pd.DataFrame(columns=['Date', 'Team', 'Acquired', 'Relinquished', 'Notes'])

for row in gleague_table.find_all('tr'):
    columns = row.find_all('td')
    
    if(columns != []):
        date = columns[0].text.strip()
        team = columns[1].text.strip()
        acq = columns[2].text.strip()
        rel = columns[3].text.strip()
        notes = columns[4].text.strip()

        df = df.append({'Date': date, 'Team': team, 'Acquired': acq, 'Relinquished': rel, 'Notes': notes}, ignore_index=True)

In [5]:
df

,Date,Team,Acquired,Relinquished,Notes
0,Date,Team,Acquired,Relinquished,Notes
1,2010-10-30,Wizards,,• Hamady Ndiaye / Hamady N'Diaye,placed on IL
2,2010-11-09,Rockets,,• Patrick Patterson,assigned to NBADL
3,2010-11-15,Raptors,,• Solomon Alabi,assigned to NBADL
4,2010-11-16,Suns,,• Gani Lawal,assigned to NBADL
5,2010-11-18,Cavaliers,,• Christian Eyenga,assigned to NBADL
6,2010-11-22,Raptors,,• Ed Davis (Adam),assigned to NBADL (for conditioning)
7,2010-11-24,Heat,,• Dexter Pittman,assigned to NBADL
8,2010-11-24,Thunder,,• Cole Aldrich,assigned to NBADL
9,2010-11-26,Nets,,• Terrence Williams,assigned to NBADL
